In [1]:
TRAIN_NUMBER = 25000
TEST_NUMBER = 10000

In [2]:
import pandas as pd

In [3]:
train = pd.read_csv("labeledTrainData.tsv", header=0, delimiter="\t", quoting=3)

In [4]:
from bs4 import BeautifulSoup

In [5]:
def clean_html_char(x):
    bs = BeautifulSoup(x)
    return bs.get_text()

In [6]:
import re
re_obj = re.compile('[^a-zA-Z]')
def keep_ascii_char(x):
    y = re_obj.sub(' ', x)
    return y

In [7]:
from nltk.corpus import stopwords
eng_stopwords = set(stopwords.words('english'))

In [8]:
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer('english')
def nltk_clean_words(x):
    ss = x.split(' ')
    return filter(lambda x: x not in eng_stopwords and x, map(lambda x: stemmer.stem(x), ss))

In [ ]:
def sentence_to_words(x):
    x = clean_html_char(x)
    x = keep_ascii_char(x)
    return nltk_clean_words(x)
def clean_review(x):
    words = sentence_to_words(x)
    return ' '.join(words)

In [ ]:
reviews = map(lambda x: clean_review(x), train['review'][:TRAIN_NUMBER])

/home/dirlt/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vectorizer = CountVectorizer(analyzer = "word", max_features = 5000)
review_mx = vectorizer.fit_transform(reviews)

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

In [ ]:
transformer = TfidfTransformer()
review_mx2 = transformer.fit_transform(review_mx)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf = RandomForestClassifier(n_estimators= 1000, oob_score= True)
rf.fit(review_mx2, train['sentiment'][:TRAIN_NUMBER])

In [ ]:
rf.oob_score_

In [ ]:
# read test data.
test = pd.read_csv('testData.tsv', header = 0, delimiter= "\t", quoting=3)
test_reviews = map(lambda x: clean_review(x), test['review'][:TEST_NUMBER])
test_mx = vectorizer.transform(test_reviews)
test_mx2 = transformer.transform(test_mx)

In [ ]:
result = rf.predict(test_mx2)

In [ ]:
output = pd.DataFrame(data = {"id": test["id"][:TEST_NUMBER], "sentiment": result})

In [ ]:
output.to_csv('bag-of-words.output.csv', index = False, quoting=3)